In [1]:
from os import getcwd
from os.path import join
import sys
sys.path.insert(0, join(getcwd(), "../module_code"))

import pandas as pd
from mlflow.tracking import MlflowClient

from models.static_models import CRRTStaticPredictor
from data.sklearn_loaders import SklearnCRRTDataModule
from main import get_mlflow_model_uri
from cli_utils import init_cli_args, load_cli_args
from data.argparse_utils import string_list_to_list, string_dict_to_dict
from data.subpopulation_utils import generate_filters, combine_filters

/home/davina/mambaforge/envs/crrt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = MlflowClient("/home/davina/Private/repos/CRRT/mlruns/")
run_id = "862e58e4cbf642bfa316ef028323f6a2"
run = client.get_run(run_id)
logged_args = run.data.tags

sys.argv = [sys.argv[0]]
load_cli_args("../options.yml")
for k,v in logged_args.items():
    if v != "None" and f"--{k}" not in sys.argv:
        sys.argv += [f"--{k}", v]
args = init_cli_args()

path = get_mlflow_model_uri(run)

model = CRRTStaticPredictor.load(args, path)
data = SklearnCRRTDataModule.from_argparse_args(args)
data.setup(args)

`use_label_encoder` is deprecated in 1.7.0.


In [14]:
args

Namespace(seed=42, experiment='static_learning', data_dir=None, ucla_crrt_data_dir='/home/davina/Private/crrt-data/UCLA/', ucla_control_data_dir='/home/davina/Private/crrt-data/UCLA/controls/', cedars_crrt_data_dir=None, preprocessed_df_file=None, serialization='parquet', controls_window_size=None, patient_age='adult', stage=None, best_model_path=None, time_interval=None, pre_start_delta={'YEARS': 0, 'MONTHS': 0, 'DAYS': 7}, post_start_delta=None, time_window_end='Start Date', slide_window_by=None, max_days_on_crrt=None, min_days_on_crrt=0, experiment_tracking=True, local_log_path='/home/davina/Private/repos/CRRT', run_name='fix-graphs', runtest=True, tune_n_trials=None, tune_metric='auroc', tune_direction='maximize', model_type='static', outcome_col_name='recommend_crrt', test_split_size=0.2, val_split_size=0.25, kbest=None, corr_thresh=None, impute_method='simple', train_val_cohort='ucla_crrt', eval_cohort='ucla_crrt', modeln='xgb', model_kwargs={}, metric_names=['ap', 'brier', 'accu

In [13]:
model.static_model.model

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [8]:
path

'/home/davina/Private/repos/CRRT/mlruns/513043940915865610/cdcbd1d31e104b448da2f3020aaae4e8/artifacts/static_model'

In [9]:
data.columns

Index(['dx_CCS_CODE_1', 'dx_CCS_CODE_10', 'dx_CCS_CODE_100', 'dx_CCS_CODE_101',
       'dx_CCS_CODE_102', 'dx_CCS_CODE_103', 'dx_CCS_CODE_104',
       'dx_CCS_CODE_105', 'dx_CCS_CODE_106', 'dx_CCS_CODE_107',
       ...
       'RACE_Black or African American', 'RACE_Multiple Races',
       'RACE_Native Hawaiian or Other Pacific Islander', 'RACE_Other',
       'RACE_Patient Refused', 'RACE_Unknown', 'RACE_White or Caucasian',
       'liver_pt_indicator', 'heart_pt_indicator', 'infection_pt_indicator'],
      dtype='object', length=11178)

In [7]:
data.train[0].shape

(2219, 11178)

In [3]:
filters = generate_filters()
dz_sex = combine_filters(filters["disease_indicator"], filters["sex"])

In [4]:
X, y = data.test
X = pd.DataFrame(X, columns=data.columns, index=y.index)
masks = {k: data.get_filter(X, *args) for k, args in dz_sex.items()}
data.test_filters = masks
model.evaluate(data, "test")

ValueError: Feature shape mismatch, expected: 13072, got 11178

In [ ]:
data.test

(array([[6.30000000e+01, 5.77634358e+01, 2.02000000e+03, ...,
         0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [3.00000000e+01, 5.77634358e+01, 2.01700000e+03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.70000000e+01, 5.77634358e+01, 2.01700000e+03, ...,
         0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        ...,
        [9.10000000e+01, 5.77634358e+01, 2.01000000e+03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.50000000e+01, 5.77634358e+01, 2.01100000e+03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.60000000e+01, 5.77634358e+01, 2.01000000e+03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 IP_PATIENT_ID                     Start Date
 00000064C94859EB973942C862550ABA  2020-08-10    0
 000008E26F8F14E5099662DDB778C698  2017-09-23    0
 00000AC44FF1345FEE4B73057620EC0F  2017-10-02    0
 000014471A50BA85E236FC7DDA593FE6  2017-11-26    0
 000015CA913525